In [62]:
import numpy as np
import pandas as pd

In [63]:
def getIndexOfColumn(df, column):
    l=df.columns.tolist()
    for i in range(len(l)):
        if l[i]==column:
            return i
    return -1
    
def mergeDFsWithColumns(firstDF, secondDF, sameColumn):
    first=firstDF.sort_values(by=[sameColumn]).values.tolist()
    second=secondDF.sort_values(by=[sameColumn]).values.tolist()
    
    idf = getIndexOfColumn(firstDF, sameColumn)
    ids = getIndexOfColumn(secondDF, sameColumn)
    
    columnsList = firstDF.columns.tolist() + secondDF.columns.tolist()[:ids] + secondDF.columns.tolist()[ids+1:]
    
    fn=len(first)
    sn=len(second)
    
    res=[]
    curr=0
    for i in range(fn):
        while curr<sn and second[curr][ids]<first[i][idf]:
            curr+=1
        if curr<sn and second[curr][ids]==first[i][idf]:
            res.append(first[i]+second[curr][:ids]+second[curr][ids+1:])
        else:
            print('One missed row')
            
    res=pd.DataFrame(data=res,columns=columnsList)
    return res

In [64]:
eventData = pd.read_csv('event.csv')
clickTrainData = pd.read_csv('click_train.csv')
clickTestData = pd.read_csv('click_test.csv')
importantColumns = ['displayId','userId']
cleanEventData = eventData[importantColumns]

In [65]:
trainDisplayId = mergeDFsWithColumns(clickTrainData, cleanEventData, 'displayId')
trainDisplayId.head()

,displayId,adId,clicked,userId
0,1210227,7182,0,782749
1,1210227,7125,0,782749
2,1210227,7181,0,782749
3,1210227,535,0,782749
4,1210227,7174,1,782749


In [66]:
testDisplayId = mergeDFsWithColumns(clickTestData, cleanEventData, 'displayId')
testDisplayId.head()

,displayId,adId,userId
0,151650,7585,10279
1,151650,6257,10279
2,151650,6690,10279
3,151938,7715,17723
4,151938,6690,17723


In [67]:
def creatingAdsWithWordsDF(adTitle, trainDisplayId):
    numberOfWords = 5
    columnsList = []
    for col in trainDisplayId.columns:
        columnsList.append(col)
    for i in range(numberOfWords):
        columnsList.append('wordId' + str(i))
        
    adTitle = adTitle.sort_values(by = ['adId']).to_numpy()
    trainForAdId = trainDisplayId.sort_values(by = ['adId']).to_numpy()

    (trainn, trainm) = trainForAdId.shape
    (adn, adm) = adTitle.shape

    (resn, resm) = (trainn, trainm + numberOfWords)

    result = np.zeros((resn, resm), dtype = np.int64)

    curr = 0
    for i in range(trainn):
        while curr < adn and adTitle[curr][0] < trainForAdId[i][1]: #1 is adId col
            curr += 1

        if curr < adn and trainForAdId[i][1] == adTitle[curr][0]:
            cnt = 0
            for j in range(0, trainm):
                result[i][cnt] = trainForAdId[i][j]
                cnt += 1
            for j in range(numberOfWords):
                if adTitle[curr + j][0] == adTitle[curr][0]:
                    result[i][cnt] = adTitle[curr + j][1]
                else:
                    result[i][cnt] = 0 #Salam bar tof!
                cnt += 1
        else:
            cnt = 0
            for j in range(0, trainm):
                result[i][cnt] = trainForAdId[i][j]
                cnt += 1
            for j in range(numberOfWords):
                result[i][cnt] = 0 #Salam bar tof!
                cnt += 1

    trainMerged = pd.DataFrame(data = result, columns = columnsList)
    trainMerged = trainMerged.sort_values(by = ['displayId'])
    return trainMerged

adTitle = pd.read_csv('ad_title.csv')
train=creatingAdsWithWordsDF(adTitle, trainDisplayId)
train.head()
#train = trainMerged.copy(deep = True)
#train.head()

,displayId,adId,clicked,userId,wordId0,wordId1,wordId2,wordId3,wordId4
3642243,1210227,7182,0,782749,108,675,734,1006,2313
3639160,1210227,7181,0,782749,853,1006,734,675,108
745066,1210227,535,0,782749,13,16,135,859,0
3625504,1210227,7125,0,782749,2544,1221,912,910,333
3636967,1210227,7174,1,782749,29,109,203,2571,2572


In [68]:
test=creatingAdsWithWordsDF(adTitle, testDisplayId)
test.head()

,displayId,adId,userId,wordId0,wordId1,wordId2,wordId3,wordId4
468220,151650,6690,10279,2487,2488,574,148,364
1117826,151650,7585,10279,57,151,202,605,0
355417,151650,6257,10279,602,76,2364,0,0
35592,151938,123,17723,21,224,340,341,0
1017470,151938,7379,17723,1849,108,1848,0,0


In [69]:
def getStandardAdImageData(adImageData):
    adImageData = adImageData.sort_values(by=['adId']).values.tolist()
    adImage=[]
    for i in range(len(adImageData)):
        l=[adImageData[i][0]]
        t=adImageData[i][1][1:-1].split(', ')
        for j in range(len(t)):
            l.append(float(t[j]))
        adImage.append(l)   
        
    columnsList=['adId']
    for i in range(1,len(adImage[0])):
        columnsList.append('imageFeature'+str(i))
    adImage=pd.DataFrame(data=adImage,columns=columnsList)
    return adImage

adImageData = pd.read_csv("ad_image.csv")
adImage=getStandardAdImageData(adImageData)
adImage.head()

,adId,imageFeature1,imageFeature2,imageFeature3,imageFeature4,imageFeature5,imageFeature6,imageFeature7,imageFeature8,imageFeature9,...,imageFeature503,imageFeature504,imageFeature505,imageFeature506,imageFeature507,imageFeature508,imageFeature509,imageFeature510,imageFeature511,imageFeature512
0,6,0.000000,0.000000,0.049760,0.407064,0.177637,0.071936,0.000000,0.000000,0.000000,...,0.123998,0.454925,0.000000,0.010478,0.205283,0.000000,0.006869,0.366349,0.359391,0.000000
1,7,0.106277,0.087542,3.344223,0.645454,1.301556,0.121954,0.000000,0.000000,0.000000,...,0.938022,0.733772,0.000000,1.211627,0.486232,0.000000,0.118570,0.079808,0.159126,0.012901
2,8,0.156233,0.000987,0.014774,0.207742,0.038036,0.011584,0.449074,0.066243,0.148653,...,0.000000,0.000000,0.000000,0.441291,0.193914,0.639011,0.046036,0.000000,0.061070,0.125147
3,11,0.042971,0.016430,0.070029,0.000000,0.562984,0.223415,1.144221,0.630494,0.123705,...,0.058076,0.099905,1.024207,0.749125,0.017977,1.022663,2.738049,0.022185,0.135937,0.710172
4,13,0.148167,0.043468,1.341706,1.226016,0.379946,0.173325,0.401060,0.134567,0.000000,...,0.693747,0.011229,0.093873,0.353185,0.941742,0.000000,0.555764,0.223577,0.071369,0.018053


# Unsupervised Clustering on image features

In [70]:
from sklearn.cluster import KMeans

In [71]:
y_pred = KMeans(n_clusters=50, random_state=23).fit_predict(adImage.drop(columns=['adId']))

In [72]:
adImage['imageClusterId'] = y_pred.tolist()
adImage.head()

,adId,imageFeature1,imageFeature2,imageFeature3,imageFeature4,imageFeature5,imageFeature6,imageFeature7,imageFeature8,imageFeature9,...,imageFeature504,imageFeature505,imageFeature506,imageFeature507,imageFeature508,imageFeature509,imageFeature510,imageFeature511,imageFeature512,imageClusterId
0,6,0.000000,0.000000,0.049760,0.407064,0.177637,0.071936,0.000000,0.000000,0.000000,...,0.454925,0.000000,0.010478,0.205283,0.000000,0.006869,0.366349,0.359391,0.000000,20
1,7,0.106277,0.087542,3.344223,0.645454,1.301556,0.121954,0.000000,0.000000,0.000000,...,0.733772,0.000000,1.211627,0.486232,0.000000,0.118570,0.079808,0.159126,0.012901,47
2,8,0.156233,0.000987,0.014774,0.207742,0.038036,0.011584,0.449074,0.066243,0.148653,...,0.000000,0.000000,0.441291,0.193914,0.639011,0.046036,0.000000,0.061070,0.125147,14
3,11,0.042971,0.016430,0.070029,0.000000,0.562984,0.223415,1.144221,0.630494,0.123705,...,0.099905,1.024207,0.749125,0.017977,1.022663,2.738049,0.022185,0.135937,0.710172,35
4,13,0.148167,0.043468,1.341706,1.226016,0.379946,0.173325,0.401060,0.134567,0.000000,...,0.011229,0.093873,0.353185,0.941742,0.000000,0.555764,0.223577,0.071369,0.018053,25


In [73]:
adImageCluster = adImage[['adId','imageClusterId']]
adImageCluster.head()

,adId,imageClusterId
0,6,20
1,7,47
2,8,14
3,11,35
4,13,25


# Chasboodane imageClusterId be dataha

In [74]:
train = mergeDFsWithColumns(train, adImageCluster, 'adId')
train.describe()

,displayId,adId,clicked,userId,wordId0,wordId1,wordId2,wordId3,wordId4,imageClusterId
count,3.642244e+06,3.642244e+06,3.642244e+06,3.642244e+06,3.642244e+06,3.642244e+06,3.642244e+06,3.642244e+06,3.642244e+06,3.642244e+06
mean,3.105835e+06,3.853480e+03,2.230227e-01,1.530336e+06,6.202482e+02,5.838237e+02,5.808164e+02,5.304167e+02,4.362280e+02,2.369640e+01
std,9.807950e+05,2.465609e+03,4.162735e-01,7.075787e+05,7.340259e+02,6.818087e+02,6.633209e+02,6.364508e+02,6.278224e+02,1.437548e+01
min,1.210227e+06,6.000000e+00,0.000000e+00,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.246184e+06,6.810000e+02,0.000000e+00,9.852190e+05,5.800000e+01,7.400000e+01,8.400000e+01,4.900000e+01,9.000000e+00,1.100000e+01
50%,3.162424e+06,5.049000e+03,0.000000e+00,1.578216e+06,2.700000e+02,2.730000e+02,2.930000e+02,2.430000e+02,1.030000e+02,2.200000e+01
75%,3.913442e+06,5.890000e+03,0.000000e+00,2.124848e+06,8.710000e+02,8.490000e+02,8.490000e+02,8.490000e+02,6.500000e+02,3.500000e+01
max,4.706267e+06,7.182000e+03,1.000000e+00,2.688642e+06,2.572000e+03,2.571000e+03,2.564000e+03,2.571000e+03,2.572000e+03,4.900000e+01


In [76]:
test = mergeDFsWithColumns(test, adImageCluster, 'adId')
test.describe()

,displayId,adId,userId,wordId0,wordId1,wordId2,wordId3,wordId4,imageClusterId
count,1.199048e+06,1.199048e+06,1.199048e+06,1.199048e+06,1.199048e+06,1.199048e+06,1.199048e+06,1.199048e+06,1.199048e+06
mean,8.076686e+05,5.846056e+03,3.904691e+05,6.720492e+02,5.292089e+02,5.864587e+02,5.947925e+02,5.140155e+02,2.297698e+01
std,3.033041e+05,2.381880e+03,2.206594e+05,8.582955e+02,6.890941e+02,7.357560e+02,7.347888e+02,7.824366e+02,1.520842e+01
min,1.516500e+05,8.000000e+00,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.433820e+05,5.917000e+03,1.928080e+05,5.100000e+01,6.500000e+01,7.900000e+01,4.900000e+01,9.000000e+00,9.000000e+00
50%,8.249250e+05,7.002000e+03,3.952770e+05,2.020000e+02,2.140000e+02,2.300000e+02,2.470000e+02,8.600000e+01,2.000000e+01
75%,1.075397e+06,7.257000e+03,5.821950e+05,1.026000e+03,6.990000e+02,7.340000e+02,8.570000e+02,7.550000e+02,3.500000e+01
max,1.328976e+06,7.867000e+03,7.817900e+05,2.679000e+03,2.681000e+03,2.677000e+03,2.678000e+03,2.683000e+03,4.900000e+01


In [83]:
def learn(X_train, y_train, X_test, importantColumns, randomState):
    from sklearn.tree import DecisionTreeClassifier
    treeModel = DecisionTreeClassifier(random_state=randomState, max_depth=15)
    
    X_train = X_train[importantColumns]
    X_test = X_test[importantColumns]
    
    treeModel.fit(X_train, y_train)
    
    predictions = treeModel.predict_proba(X_test)
    return predictions

In [78]:
# Split must has displayId, clicked
# outputs [X_train, y_train, X_test, y_test]

def split(train, random_state):
    train=train.sort_values(by='displayId')
    i = 0
    columnsList = []
    for col in train.columns:
        if col == 'displayId':
            displayId = i
            columnsList.append(col)
        elif col == 'clicked':
            clickedId = i
        else:
            columnsList.append(col)
        i += 1            
    train2 = train.values.tolist()
    i = 0
    y_train = []
    X_train = []
    y_test = []
    X_test = []
    while(i < len(train2)):
        k = i + 1
        while(k < len(train2) and train2[i][displayId] == train2[k][displayId]):
            k += 1
        if(np.random.randint(low = 1, high = 100) < 33):
            for j in range(i, k):
                y_test.append(train2[j][clickedId])
                X_test.append(train2[j][:clickedId]+train2[j][clickedId + 1:])
        else:
            for j in range(i, k):
                y_train.append(train2[j][clickedId])
                X_train.append(train2[j][:clickedId]+train2[j][clickedId + 1:])
        i = k
        
    X_train=pd.DataFrame(data=X_train,columns=columnsList)
    X_test=pd.DataFrame(data=X_test,columns=columnsList)
    y_train=pd.DataFrame(data=y_train,columns=['clicked'])
    y_test=pd.DataFrame(data=y_test,columns=['clicked'])
    res = [X_train, y_train, X_test, y_test]
    return res

In [79]:
def score(X_test, y_test, predictions):
    predictions=pd.DataFrame(data=predictions,columns=['tmp','probability'])
    predictions=predictions[['probability']]
    result = pd.concat([X_test, y_test, predictions], axis=1, sort=False)
    result = result.sort_values(by=['displayId','probability'])
    importantCols=['displayId','clicked']
    result=result[importantCols]
    result=result.values.tolist()
   
    ans=0.0
    t=0
    rank=1
    for i in range(len(result)):
        if i==0 or result[i-1][0]!=result[i][0]:
            t+=1
            rank=1
        ans+=result[i][1]/(1.0*rank)
        rank+=1
    ans*=100.0
    ans/=t
    return ans

In [80]:
def rankList(probabilities):
    ans = []
    n = len(probabilities)
    for i in range(n):
        cnt = 1
        for j in range(n):
            if i == j:
                continue
            if probabilities[i] < probabilities[j] or (probabilities[i] == probabilities[j] and j < i):
                cnt += 1
        ans.append(cnt)
    return ans

def getRankColumn(predictions, test):   
    i = 0
    for col in test.columns:
        if col == 'displayId':
            displaydId = i
        elif col == 'adId':
            adId = i
        i += 1
    
    testData = pd.read_csv('click_test.csv').to_numpy()
    testPre = test.to_numpy()

    (n, m) = testData.shape
    (tn, tm) = testPre.shape

    rank = []

    i = 0

    l = 0
    while l < tn:
        r = l + 1
        while r < tn and testPre[r][displaydId] == testPre[l][displaydId]:
            r += 1
        List = []
        lki = i
        while i < n and testData[i][0] == testPre[l][displaydId]:
            for j in range(l, r):
                if testPre[j][adId] == testData[i][1]:
                    List.append(predictions[j][1])
                    break
            i += 1
        rankListed = rankList(List)
        for j in range(lki, i):
            rank.append(rankListed[j - lki])
        l = r
    return rank

In [81]:
def calc(train, importantColumns):
    [X_train, y_train, X_test, y_test] = split(train, 23) #23 is random_state
    predictions = learn(X_train, y_train, X_test, importantColumns, 23)
    print(score(X_test, y_test, predictions))

In [54]:
def getCsv(train, test, importantColumns, nameStr):
    test = test.sort_values(by = ['displayId'])
    X_train = train.drop(columns=['clicked'])
    y_train = train['clicked']
    predictions = learn(X_train, y_train, test, importantColumns, 23)
    
    print(len(predictions))
    
    rank = getRankColumn(predictions, test)
    
    print(len(rank))
    
    testData = pd.read_csv('click_test.csv')
    
    print(testData.shape)
    
    testData['clicked'] = rank
    testData.to_csv(nameStr,index=False)

In [85]:
importantColumns = ['userId', 'wordId0', 'wordId1', 'wordId2', 'wordId3', 'imageClusterId']
calc(train, importantColumns)

45.70763274467495


In [51]:
test.head()

,displayId,adId,userId,wordId0,wordId1,wordId2,wordId3,wordId4,imageClusterId
0,816736,8,432827,8,32,36,37,38,14
1,710459,8,356275,8,32,36,37,38,14
2,322227,8,62861,8,32,36,37,38,14
3,811851,8,429354,8,32,36,37,38,14
4,607152,8,280016,8,32,36,37,38,14


In [57]:
importantColumns = ['userId', 'wordId0', 'wordId1', 'wordId2', 'wordId3', 'wordId4', 'imageClusterId']
getCsv(train, test, importantColumns, 'imageCluserIdAns.csv')

1199048
1199048
(1199048, 2)
